In [13]:

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import joblib
from sklearn.metrics import average_precision_score, roc_auc_score, ndcg_score
import hopsworks
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import joblib
from sklearn.metrics import average_precision_score, roc_auc_score, ndcg_score

# Login to Hopsworks and get project
project = hopsworks.login()
mr = project.get_model_registry()


2025-04-20 12:03:29,351 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-20 12:03:29,718 INFO: Initializing external client
2025-04-20 12:03:29,724 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-20 12:03:30,641 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-20 12:03:31,553 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


In [14]:

fs = project.get_feature_store()
# Load from Hopsworks feature groups
users_fg = fs.get_feature_group(name="users", version=1)
events_fg = fs.get_feature_group(name="events", version=1)
interactions_fg = fs.get_feature_group(name="interactions", version=1)

In [15]:
users_df = users_fg.read()


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.79s) 


In [31]:
interactions_df.head()

,interaction_id,user_id,event_id,interaction_type,interaction_time,interaction_distance_to_event
0,28f8230c-a22c-4827-b808-5bccd8207cb1,IY531R,WH966B,no,2023-04-17 01:38:57.844458+00:00,5736.00000
1,4140ef03-afce-4fe5-a9a9-8e43fd45dfaa,LT819S,OI841N,invited,2024-05-28 12:43:38.312350+00:00,7630.00000
2,BI623H,UP287J,IO571I,yes,2025-08-21 22:51:24.898618+00:00,14.52132
3,08e342ae-f9b5-4358-9f41-cf89671b9ae2,RG723K,YW813D,invited & no,2023-05-22 11:28:30.845737+00:00,7007.00000
4,7cfbb86b-6d7a-4f1a-b05f-ffa78bc41ffb,DM205I,RM794Y,invited & no,2023-10-31 00:39:12.438860+00:00,6019.00000


Users_data:

user_id	user_lat	user_lon	user_city	user_weather_preference	age	user_interests	signup_date	social_connectedness
0	JI904B	36.122607	138.598711	Tokyo	outdoor	27	music	2023-07-24 10:35:22.959064+00:00	11
1	YF384K	-33.897451	151.157340	Sydney	any	26	music cinema sports	2023-04-24 20:46:40.735141+00:00	17
2	RJ912O	40.593968	-72.327571	New York	indoor	24	travel	2023-10-13 11:37:21.333403+00:00	16

events_data:
event_id	title	event_type	event_lat	event_lon	event_city	start_time	duration	weather_condition	temperature	attendance_rate	event_indoor_capability
0	CA178V	seamless leadingedge timeframe arts culture in...	Arts & Culture	44.019579	-80.891209	Toronto	2025-06-14 10:23:15.662617+00:00	180	Clear	12.3	5.226955	True
1	TF630U	visionary maximized definition community cause...	Community & Causes	40.812661	-74.051607	New York	2025-04-13 14:02:14.486362+00:00	360	Clear	18.7	8.091905	False
2	DY324Y	upsized local function business networking in ...	Business & Networking	40.747403	-73.953319	New York	2025-05-17 14:11:30.484025+00:00	240	Cloudy	16.9	31.945106	True

interactions_data:
interaction_id	user_id	event_id	interaction_type	interaction_time	interaction_distance_to_event
0	28f8230c-a22c-4827-b808-5bccd8207cb1	IY531R	WH966B	no	2023-04-17 01:38:57.844458+00:00	5736.00000
1	4140ef03-afce-4fe5-a9a9-8e43fd45dfaa	LT819S	OI841N	invited	2024-05-28 12:43:38.312350+00:00	7630.00000
2	BI623H	UP287J	IO571I	yes	2025-08-21 22:51:24.898618+00:00	14.52132
3	08e342ae-f9b5-4358-9f41-cf89671b9ae2	RG723K	YW813D	invited & no	2023-05-22 11:28:30.845737+00:00	7007.00000
4	7cfbb86b-6d7a-4f1a-b05f-ffa78bc41ffb	DM205I	RM794Y	invited & no	2023-10-31 00:39:12.438860+00:00	6019.00000

# Define features for query and candidate towers (retrieval model only)
QUERY_FEATURES = ["user_id", "user_city", "age", "user_interests"]
CANDIDATE_FEATURES = ["event_id", "event_type", "event_city"]


# Create Ranking Feature group
events_interactions_df = pd.merge(
    interactions_df, 
    events_df, 
    on='event_id', 
    how='inner',
    suffixes=('', '_event')  # Add suffix for event columns
)

ranking_df = pd.merge(
    events_interactions_df, 
    users_df, 
    on='user_id', 
    how='inner',
    suffixes=('', '_user')  # Add suffix for user columns
)

ranking_df['label'] = ranking_df['interaction_type'].apply(
    lambda x: 1 if x in ['maybe', 'invited & maybe', 'yes', 'invited & yes'] else 0
)

ranking_df_with_weather = ranking_df.drop(
    ['interaction_id', 'interaction_type','interaction_time',\
     'start_time', 'signup_date','social_connectedness'], 
    axis=1, 
    inplace=True,
)

ranking_fg_weather = fs.get_or_create_feature_group(
    name="ranking-with-weather",
    description="Ranking Data with weather conditions.",
    version=1,
    primary_key=["user_id", "video_id"],
    online_enabled=True,
)

ranking_fg_weather.insert(ranking_df_with_weather)
print('Done ✅')

# create ranking data without weather information
ranking_df_without_weather = ranking_df.drop(['interaction_id', 'interaction_type',
       'interaction_time', 'start_time','weather_condition',
       'temperature', 'weather_preference',
       'signup_date', 'social_connectedness'])

ranking_fg_without_weather = fs.get_or_create_feature_group(
    name="ranking-without-weather",
    description="Ranking Data without weather conditions.",
    version=1,
    primary_key=["user_id", "video_id"],
    online_enabled=True,
)

ranking_fg_without_weather.insert(ranking_df_without_weather)
print('Done ✅')

print("Feature backfill")

In [16]:
events_df = events_fg.read()
interactions_df = interactions_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.87s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.19s) 


In [22]:
# Load models
query_model = mr.get_model("query_model", version=1).download()
candidate_model = mr.get_model("candidate_model", version=1).download()
weather_ranking_model = mr.get_model("weather_ranking_model", version=1).download()
no_weather_ranking_model = mr.get_model("no_weather_ranking_model", version=1).download()

In [18]:

# For each user, generate top-K candidate events using the retrieval model
def generate_top_k_candidates(user_df, k=100):
    user_ds = tf.data.Dataset.from_tensor_slices({
        "user_id": user_df["user_id"].values,
        "user_city": user_df["user_city"].values,
        "age": user_df["age"].values,
        "user_index": user_df.index.values
    }).batch(128)
    
    user_embs = query_model(user_ds)
    event_ds = tf.data.Dataset.from_tensor_slices({
        "event_id": events_df["event_id"].values,
        "event_city": events_df["event_city"].values,
        "event_type": events_df["event_type"].values,
        "event_index": events_df.index.values
    }).batch(128)
    
    event_embs = candidate_model(event_ds)
    
    scores = tf.linalg.matmul(user_embs, event_embs, transpose_b=True)
    top_k_idx = tf.math.top_k(scores, k=k).indices.numpy()
    
    candidates = []
    for u_idx, event_indices in enumerate(top_k_idx):
        user_id = user_df.iloc[u_idx]["user_id"]
        for idx in event_indices:
            event_id = events_df.iloc[idx]["event_id"]
            candidates.append((user_id, event_id))
    
    return pd.DataFrame(candidates, columns=["user_id", "event_id"])


In [19]:

# Label = 1 if (user, event) in interactions
def create_ranking_dataset(candidates_df):
    interactions_set = set(zip(interactions_df["user_id"], interactions_df["event_id"]))
    candidates_df["label"] = candidates_df.apply(
        lambda row: 1 if (row["user_id"], row["event_id"]) in interactions_set else 0, axis=1)
    
    df = candidates_df.merge(users_df, on="user_id").merge(events_df, on="event_id")
    return df


In [20]:

def evaluate_weather_ranking_model(ranking_df):
    y_true = ranking_df["label"].astype(int).values
    X = ranking_df.drop(columns=["label", "user_id", "event_id"])
    y_score = weather_ranking_model.predict_proba(X)[:, 1]
    
    metrics = {
        "AUC": roc_auc_score(y_true, y_score),
        "MAP": average_precision_score(y_true, y_score)
    }
    
    for k in [5, 10, 20]:
        sorted_idx = np.argsort(-y_score)
        relevance = y_true[sorted_idx]
        top_k = relevance[:k]
        metrics[f"Precision@{k}"] = top_k.mean()
        metrics[f"Recall@{k}"] = top_k.sum() / max(np.sum(relevance), 1)
        metrics[f"NDCG@{k}"] = ndcg_score([relevance], [y_score], k=k)
    
    return metrics


In [23]:

# Run pipeline
top_k_candidates = generate_top_k_candidates(users_df)
ranking_df = create_ranking_dataset(top_k_candidates)
results = evaluate_weather_ranking_model(ranking_df)

for metric, score in results.items():
    print(f"{metric}: {score:.4f}")


TypeError: 'str' object is not callable

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import joblib
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import hopsworks
from geopy.distance import great_circle

# Initialize Hopsworks connection
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()

# Load feature groups
users_fg = fs.get_feature_group(name="users", version=1)
events_fg = fs.get_feature_group(name="events", version=1)
interactions_fg = fs.get_feature_group(name="interactions", version=1)

# Read data with specific columns
users_df = users_fg.read(online=False)[['user_id', 'city', 'age', 'interests']]
events_df = events_fg.read(online=False)[['event_id', 'type', 'city', 'datetime', 'latitude', 'longitude']]
interactions_df = interactions_fg.read(online=False)[['user_id', 'event_id', 'timestamp']]

# Load models
query_model = mr.get_model("query_model", version=1).download()
candidate_model = mr.get_model("candidate_model", version=1).download()
weather_ranking_model = mr.get_model("weather_ranking_model", version=1).download()
no_weather_ranking_model = mr.get_model("no_weather_ranking_model", version=1).download()

# Feature preprocessing
event_type_encoder = LabelEncoder().fit(events_df['type'])
user_city_encoder = LabelEncoder().fit(users_df['city'])
event_city_encoder = LabelEncoder().fit(events_df['city'])

def calculate_distance(row):
    user_coords = (row['user_lat'], row['user_lon'])
    event_coords = (row['event_lat'], row['event_lon'])
    return great_circle(user_coords, event_coords).km

def generate_top_k_candidates(user_df, k=100):
    # Create TensorFlow datasets
    user_ds = tf.data.Dataset.from_tensor_slices({
        "user_id": user_df["user_id"].values,
        "city": user_city_encoder.transform(user_df["city"]),
        "age": user_df["age"].values.astype(np.float32)
    }).batch(128)
    
    event_ds = tf.data.Dataset.from_tensor_slices({
        "event_id": events_df["event_id"].values,
        "type": event_type_encoder.transform(events_df["type"]),
        "city": event_city_encoder.transform(events_df["city"])
    }).batch(128)
    
    # Generate embeddings
    user_embs = query_model(user_ds)
    event_embs = candidate_model(event_ds)
    
    # Calculate scores and get top K
    scores = tf.linalg.matmul(user_embs, event_embs, transpose_b=True)
    top_k_indices = tf.math.top_k(scores, k=k).indices.numpy()
    
    # Create candidate DataFrame
    candidates = []
    for user_idx, event_indices in enumerate(top_k_indices):
        user_id = user_df.iloc[user_idx]["user_id"]
        for event_idx in event_indices:
            event_id = events_df.iloc[event_idx]["event_id"]
            candidates.append((user_id, event_id))
    
    return pd.DataFrame(candidates, columns=["user_id", "event_id"])

def create_ranking_dataset(candidates_df):
    # Create labels
    interactions_set = set(zip(interactions_df["user_id"], interactions_df["event_id"]))
    candidates_df["label"] = candidates_df.apply(
        lambda row: 1 if (row["user_id"], row["event_id"]) in interactions_set else 0, axis=1
    )
    
    # Merge with user and event features
    df = candidates_df.merge(
        users_df.rename(columns={'city': 'user_city', 'latitude': 'user_lat', 'longitude': 'user_lon'}),
        on="user_id"
    ).merge(
        events_df.rename(columns={'city': 'event_city', 'type': 'event_type', 
                                'latitude': 'event_lat', 'longitude': 'event_lon'}),
        on="event_id"
    )
    
    # Calculate interaction distance
    df['interaction_distance_to_event'] = df.apply(calculate_distance, axis=1)
    
    # Convert categorical features
    df['event_type'] = event_type_encoder.transform(df['event_type'])
    df['user_city'] = user_city_encoder.transform(df['user_city'])
    df['event_city'] = event_city_encoder.transform(df['event_city'])
    
    return df

def evaluate_ranking_model(model, ranking_df):
    # Prepare features
    features = ranking_df[['interaction_distance_to_event', 'event_type', 
                          'user_city', 'event_city', 'age']]
    
    # Generate predictions
    y_score = model.predict_proba(features)[:, 1]
    y_true = ranking_df["label"].values
    
    # Calculate metrics
    metrics = {
        "AUC": roc_auc_score(y_true, y_score),
        "MAP": average_precision_score(y_true, y_score)
    }
    
    # Calculate top-K metrics
    for k in [5, 10, 20]:
        top_k_mask = np.argsort(-y_score)[:k]
        top_k_labels = y_true[top_k_mask]
        
        metrics[f"Precision@{k}"] = top_k_labels.mean()
        metrics[f"Recall@{k}"] = top_k_labels.sum() / max(y_true.sum(), 1)
        
        # Calculate NDCG
        ideal_sorted = np.sort(y_true)[::-1]
        dcg = sum((2**rel - 1) / np.log2(i+2) for i, rel in enumerate(top_k_labels))
        idcg = sum((2**rel - 1) / np.log2(i+2) for i, rel in enumerate(ideal_sorted[:k]))
        metrics[f"NDCG@{k}"] = dcg / idcg if idcg > 0 else 0
    
    return metrics

# Execute pipeline
print("Generating candidates...")
top_k_candidates = generate_top_k_candidates(users_df)

print("Creating ranking dataset...")
ranking_df = create_ranking_dataset(top_k_candidates)

print("Evaluating weather ranking model...")
weather_results = evaluate_ranking_model(weather_ranking_model, ranking_df)

print("\nWeather Model Metrics:")
for metric, score in weather_results.items():
    print(f"{metric}: {score:.4f}")

print("\nEvaluating no-weather ranking model...")
no_weather_results = evaluate_ranking_model(no_weather_ranking_model, ranking_df)

print("\nNo-Weather Model Metrics:")
for metric, score in no_weather_results.items():
    print(f"{metric}: {score:.4f}")


---

---

In [25]:

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from sklearn.metrics import average_precision_score, roc_auc_score, ndcg_score
from sklearn.preprocessing import LabelEncoder
from geopy.distance import great_circle
import hopsworks


In [26]:

# Hopsworks connection
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()



2025-04-20 12:28:42,378 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-20 12:28:42,412 INFO: Initializing external client
2025-04-20 12:28:42,413 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-04-20 12:28:43,811 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


In [28]:


# Load models
query_model = mr.get_model("query_model", version=1).load()
candidate_model = mr.get_model("candidate_model", version=1).load()
weather_ranking_model = mr.get_model("weather_ranking_model", version=1).load()
no_weather_ranking_model = mr.get_model("no_weather_ranking_model", version=1).load()



AttributeError: 'Model' object has no attribute 'load'

---

---

In [32]:
import hopsworks
import pandas as pd
import numpy as np
import tensorflow as tf
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import ndcg_score
from hsml.schema import Schema
from hsml.model_schema import ModelSchema


In [33]:

# Initialize Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()

# Load feature views
weather_rank_fv = fs.get_feature_view("weather_ranking", version=1)
no_weather_rank_fv = fs.get_feature_view("no_weather_ranking", version=1)


2025-04-20 13:40:07,781 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-20 13:40:07,847 INFO: Initializing external client
2025-04-20 13:40:07,854 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-20 13:40:08,525 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-20 13:40:09,509 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


In [ ]:

# Constants
EMBED_DIM = 32
RETRIEVAL_K = 100
RANKING_K = 10

import tensorflow as tf
from tensorflow.keras.layers import TFSMLayer
import joblib
import os

def load_models():
    """Load models from Hopsworks Model Registry"""
    project = hopsworks.login()
    mr = project.get_model_registry()
    
    # Load retrieval components using TFSMLayer
    query_model = tf.keras.layers.TFSMLayer(
        mr.get_model("query_model", version=1).download(),
        call_endpoint='serving_default'
    )
    
    candidate_model = tf.keras.layers.TFSMLayer(
        mr.get_model("candidate_model", version=1).download(),
        call_endpoint='serving_default'
    )
    
    # Load CatBoost models with explicit file paths
    weather_model_path = mr.get_model("weather_ranking_model", version=2).download()
    weather_model = joblib.load(os.path.join(weather_model_path, "weather_ranking_model.pkl"))
    
    no_weather_model_path = mr.get_model("no_weather_ranking_model", version=2).download()
    no_weather_model = joblib.load(os.path.join(no_weather_model_path, "no_weather_ranking_model.pkl"))
    
    return query_model, candidate_model, weather_model, no_weather_model




def generate_candidates(query_model, candidate_model, users_df, events_df, k=RETRIEVAL_K):
    """Generate top-K candidates using Two-Tower model"""
    # Create TensorFlow datasets
    user_ds = tf.data.Dataset.from_tensor_slices({
        "user_id": users_df["user_id"],
        "user_city": users_df["user_city"],
        "age": users_df["age"].astype(np.float32),
        "user_interests": users_df["user_interests"]
    }).batch(128)
    
    event_ds = tf.data.Dataset.from_tensor_slices({
        "event_id": events_df["event_id"],
        "event_type": events_df["event_type"],
        "event_city": events_df["event_city"]
    }).batch(128)
    
    # Generate embeddings
    user_embs = query_model(user_ds)
    event_embs = candidate_model(event_ds)
    
    # Calculate scores and get top K
    scores = tf.linalg.matmul(user_embs, event_embs, transpose_b=True)
    top_k_idx = tf.math.top_k(scores, k=k).indices.numpy()
    
    # Create candidate DataFrame
    candidates = []
    for user_idx, event_indices in enumerate(top_k_idx):
        user_id = users_df.iloc[user_idx]["user_id"]
        for event_idx in event_indices:
            event_id = events_df.iloc[event_idx]["event_id"]
            candidates.append((user_id, event_id))
    
    return pd.DataFrame(candidates, columns=["user_id", "event_id"])

def rank_candidates(ranking_model, candidates_df, feature_view):
    """Rank candidates using CatBoost model"""
    # Get ranking features
    features = feature_view.get_batch_data(candidates_df)
    
    # Generate predictions
    if 'weather_condition' in features.columns:  # Weather model
        features = features[['interaction_distance_to_event', 'event_type', 
                            'user_city', 'event_city', 'age', 'weather_condition',
                            'temperature', 'attendance_rate']]
    else:  # No-weather model
        features = features[['interaction_distance_to_event', 'event_type',
                            'user_city', 'event_city', 'age', 'attendance_rate']]
    
    rank_pool = Pool(features, cat_features=features.select_dtypes(include=["object"]).columns.tolist())
    return ranking_model.predict_proba(rank_pool)[:, 1]

def evaluate_ranking(y_true, y_score, k=RANKING_K):
    """Calculate ranking metrics including NDCG@k"""
    metrics = {
        "AUC": roc_auc_score(y_true, y_score),
        "Average Precision": average_precision_score(y_true, y_score)
    }
    
    sorted_idx = np.argsort(-y_score)
    for k in [5, 10, 20]:
        top_k = y_true[sorted_idx][:k]
        metrics[f"NDCG@{k}"] = ndcg_score([y_true], [y_score], k=k)
        metrics[f"Precision@{k}"] = top_k.mean()
        metrics[f"Recall@{k}"] = top_k.sum() / max(y_true.sum(), 1)
    
    return metrics


In [55]:

# Load data and models
users_df = fs.get_feature_group("users", 1).read()
events_df = fs.get_feature_group("events", 1).read()
query_model, candidate_model, weather_model, no_weather_model = load_models()



Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.77s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.42s) 
2025-04-20 14:53:13,076 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-20 14:53:13,089 INFO: Initializing external client
2025-04-20 14:53:13,090 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-20 14:53:13,575 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-20 14:53:14,349 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


In [ ]:

# Generate candidates
candidates_df = generate_top_k_candidates(user_df, k=100)

# Create ranking dataset
ranking_df = weather_rank_fv.get_batch_data(candidates_df)
no_weather_df = no_weather_rank_fv.get_batch_data(candidates_df)

# Rank candidates
weather_scores = rank_candidates(weather_model, ranking_df, weather_rank_fv)
no_weather_scores = rank_candidates(no_weather_model, no_weather_df, no_weather_rank_fv)

# Get ground truth
interactions = fs.get_feature_group("interactions", 1).read()
y_true = candidates_df.merge(interactions, on=["user_id", "event_id"], how="left")["label"].fillna(0)

# Evaluate both models
print("Weather Model Metrics:")
print(evaluate_ranking(y_true, weather_scores))

print("\nNo-Weather Model Metrics:")
print(evaluate_ranking(y_true, no_weather_scores))


In [ ]:
/home/nkama/masters_thesis_project/thesis/sythesize_from_scratch/no_weather_ranking_model.pkl
/home/nkama/masters_thesis_project/thesis/sythesize_from_scratch/weather_ranking_model.pkl
/home/nkama/masters_thesis_project/thesis/sythesize_from_scratch/retrieve_rank_evaluate_pipeline.ipynb

In [69]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
from sklearn.metrics import average_precision_score, roc_auc_score, ndcg_score
from sklearn.preprocessing import LabelEncoder
from catboost import Pool
import hopsworks

# Constants
EMBED_DIM = 32
RETRIEVAL_K = 100
RANKING_K = 10


from pathlib import Path

def load_models():
    """Load models from Hopsworks Model Registry"""
    project = hopsworks.login()
    mr = project.get_model_registry()
    
    # Load TensorFlow SavedModels using TFSMLayer
    query_model_path = mr.get_model("query_model", version=1).download()
    query_model = tf.keras.layers.TFSMLayer(
        query_model_path,
        call_endpoint='serving_default'
    )
    
    candidate_model_path = mr.get_model("candidate_model", version=1).download()
    candidate_model = tf.keras.layers.TFSMLayer(
        candidate_model_path,
        call_endpoint='serving_default'
    )
    
    # Load CatBoost models with explicit paths
    def load_catboost(model_name, version):
        model_path = Path(mr.get_model(model_name, version=version).download())
        model_file = next((f for f in model_path.iterdir() if f.suffix == '.pkl'), None)
        if not model_file:
            raise ValueError(f"No .pkl file found in {model_path}")
        return joblib.load(model_path / model_file)
    
    weather_model = load_catboost("weather_ranking_model", 1)
    no_weather_model = load_catboost("no_weather_ranking_model", 1)
    
    return query_model, candidate_model, weather_model, no_weather_model


def generate_candidates(query_model, candidate_model, users_df, events_df, k=RETRIEVAL_K):
    """Generate top-K candidates using Two-Tower model"""
    # Create TensorFlow datasets with original feature names
    user_ds = tf.data.Dataset.from_tensor_slices({
        "user_id": users_df["user_id"].values,
        "user_city": users_df["user_city"].values,
        "age": users_df["age"].values.astype(np.float32),
        "user_interests": users_df["user_interests"].values
    }).batch(128)
    
    event_ds = tf.data.Dataset.from_tensor_slices({
        "event_id": events_df["event_id"].values,
        "event_type": events_df["event_type"].values,
        "event_city": events_df["event_city"].values
    }).batch(128)
    
    # Generate embeddings using explicit keyword arguments
    user_embs = []
    for batch in user_ds:
        user_embs.append(query_model(
            user_id=batch['user_id'],
            user_city=batch['user_city'],
            age=batch['age'],
            user_interests=batch['user_interests']
        ))
    user_embs = tf.concat(user_embs, axis=0)
    
    event_embs = []
    for batch in event_ds:
        event_embs.append(candidate_model(
            event_id=batch['event_id'],
            event_type=batch['event_type'],
            event_city=batch['event_city']
        ))
    event_embs = tf.concat(event_embs, axis=0)
    
    # Rest of the function remains unchanged
    scores = tf.linalg.matmul(user_embs, event_embs, transpose_b=True)
    top_k_idx = tf.math.top_k(scores, k=k).indices.numpy()
    
    candidates = []
    for user_idx, event_indices in enumerate(top_k_idx):
        user_id = users_df.iloc[user_idx]["user_id"]
        candidates.extend([(user_id, events_df.iloc[idx]["event_id"]) for idx in event_indices])
    
    return pd.DataFrame(candidates, columns=["user_id", "event_id"])




def rank_candidates(ranking_model, candidates_df, feature_view):
    """Rank candidates with proper feature preprocessing"""
    # Get ranking features
    features = feature_view.get_batch_data(candidates_df)
    
    # Handle missing weather features
    if 'weather_condition' in features.columns:
        features = features[['interaction_distance_to_event', 'event_type', 
                           'user_city', 'event_city', 'age', 'weather_condition',
                           'temperature', 'attendance_rate']].fillna(method='ffill')
    else:
        features = features[['interaction_distance_to_event', 'event_type',
                           'user_city', 'event_city', 'age', 'attendance_rate']]
    
    # Convert categorical features
    cat_features = features.select_dtypes(include=["object"]).columns.tolist()
    return ranking_model.predict_proba(Pool(features, cat_features=cat_features))[:, 1]

def evaluate_ranking(y_true, y_score, k_values=[5, 10, 20]):
    """Calculate ranking metrics with proper NDCG calculation"""
    metrics = {
        "AUC": roc_auc_score(y_true, y_score),
        "MAP": average_precision_score(y_true, y_score)
    }
    
    sorted_idx = np.argsort(-y_score)
    for k in k_values:
        top_k = y_true[sorted_idx][:k]
        metrics[f"Precision@{k}"] = top_k.mean()
        metrics[f"Recall@{k}"] = top_k.sum() / max(y_true.sum(), 1)
        metrics[f"NDCG@{k}"] = ndcg_score([y_true], [y_score], k=k)
    
    return metrics



# Initialize Hopsworks connection
project = hopsworks.login()
fs = project.get_feature_store()

# Load data
users_df = fs.get_feature_group("users", 1).read()
events_df = fs.get_feature_group("events", 1).read()
interactions_df = fs.get_feature_group("interactions", 1).read()

# Load models
query_model, candidate_model, weather_model, no_weather_model = load_models()


# Generate candidates
candidates_df = generate_candidates(query_model, candidate_model, users_df, events_df)


# Create ranking datasets
weather_rank_fv = fs.get_feature_view("weather_ranking", 2)
no_weather_rank_fv = fs.get_feature_view("no_weather_ranking", 2)

# Get ground truth labels
interactions_set = set(zip(interactions_df["user_id"], interactions_df["event_id"]))
candidates_df["label"] = candidates_df.apply(
    lambda row: 1 if (row["user_id"], row["event_id"]) in interactions_set else 0, axis=1)

# Evaluate both models
print("Evaluating Weather Model...")
weather_scores = rank_candidates(weather_model, candidates_df, weather_rank_fv)
print(evaluate_ranking(candidates_df["label"].values, weather_scores))

print("\nEvaluating No-Weather Model...")
no_weather_scores = rank_candidates(no_weather_model, candidates_df, no_weather_rank_fv)
print(evaluate_ranking(candidates_df["label"].values, no_weather_scores))


In [ ]:

# Initialize Hopsworks connection
project = hopsworks.login()
fs = project.get_feature_store()


In [ ]:

# Load data
users_df = fs.get_feature_group("users", 1).read()
events_df = fs.get_feature_group("events", 1).read()
interactions_df = fs.get_feature_group("interactions", 1).read()



In [62]:

# Load models
query_model, candidate_model, weather_model, no_weather_model = load_models()



2025-04-20 15:13:18,137 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-20 15:13:18,145 INFO: Initializing external client
2025-04-20 15:13:18,146 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-04-20 15:13:19,463 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


In [85]:

# Generate candidates
candidates_df = generate_candidates(query_model, candidate_model, users_df, events_df)


TypeError: missing a required argument: 'inputs'

In [ ]:

# Create ranking datasets
weather_rank_fv = fs.get_feature_view("weather_ranking", 2)
no_weather_rank_fv = fs.get_feature_view("no_weather_ranking", 2)

# Get ground truth labels
interactions_set = set(zip(interactions_df["user_id"], interactions_df["event_id"]))
candidates_df["label"] = candidates_df.apply(
    lambda row: 1 if (row["user_id"], row["event_id"]) in interactions_set else 0, axis=1)

# Evaluate both models
print("Evaluating Weather Model...")
weather_scores = rank_candidates(weather_model, candidates_df, weather_rank_fv)
print(evaluate_ranking(candidates_df["label"].values, weather_scores))

print("\nEvaluating No-Weather Model...")
no_weather_scores = rank_candidates(no_weather_model, candidates_df, no_weather_rank_fv)
print(evaluate_ranking(candidates_df["label"].values, no_weather_scores))


In [83]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TFSMLayer
from catboost import Pool
import hopsworks
import joblib

class RecommendationSystem:
    def __init__(self):
        # Initialize Hopsworks connections
        self.project = hopsworks.login()
        self.fs = self.project.get_feature_store()
        self.mr = self.project.get_model_registry()
        
        # Load feature views
        self.users_fv = self.fs.get_feature_view("users", 1)
        self.events_fv = self.fs.get_feature_view("events", 1)
        self.interactions_fv = self.fs.get_feature_group("interactions", 1)
        self.candidate_embeddings_fv = self.fs.get_feature_view("events_candidate_embeddings", 1)
        
        # Load models
        self.query_model = self.load_tf_model("query_model", 1)
        self.candidate_model = self.load_tf_model("candidate_model", 1)
        self.weather_model = self.load_catboost("weather_ranking_model", 1)
        self.no_weather_model = self.load_catboost("no_weather_ranking_model", 1)

    def load_tf_model(self, name, version):
        model_path = self.mr.get_model(name, version).download()
        return TFSMLayer(model_path, call_endpoint='serving_default')

    # Load CatBoost models with explicit paths
    def load_catboost(self, model_name, version):
        model_path = Path(mr.get_model(model_name, version=version).download())
        model_file = next((f for f in model_path.iterdir() if f.suffix == '.pkl'), None)
        if not model_file:
            raise ValueError(f"No .pkl file found in {model_path}")
        return joblib.load(model_path / model_file)
    
   

    def generate_candidates(self, user_id, k=100):
        # Get user features
        user_features = self.users_fv.get_feature_vector({"user_id": user_id})

        # Generate user embedding using explicit keyword arguments
        user_embedding = self.query_model(
            user_id=user_features['user_id'],
            user_city=user_features['user_city'],
            age=user_features['age'],
            user_interests=user_features['user_interests']
        )

        # Retrieve candidate event IDs from interactions
        seen_events = self.interactions_fv.get_feature_vector(
            {"user_id": user_id}, 
            return_type="pandas"
        )["event_id"].tolist()

        # Create entry list for candidate embeddings using event_id
        candidate_entries = [
            {"event_id": event_id} 
            for event_id in self.events_fv.get_feature_vectors().event_id
            if event_id not in seen_events
        ]

        # Retrieve candidate embeddings with proper entry parameter
        candidate_embeddings = self.candidate_embeddings_fv.get_feature_vectors(
            entry=candidate_entries,
            allow_missing=True  # Optional: handle missing entries gracefully
        )

        # Rest of the candidate generation logic
        scores = tf.linalg.matmul(user_embedding, candidate_embeddings, transpose_b=True)
        top_k_idx = tf.math.top_k(scores, k=k).indices.numpy()

        return pd.DataFrame(
            [(user_id, candidate_embeddings.iloc[idx]["event_id"]) 
                for idx in top_k_idx],
            columns=["user_id", "event_id"]
        )



    def rank_candidates(self, candidates_df, weather_data=None):
        # Get ranking features from feature store
        features = self.fs.get_batch_data(candidates_df)
        
        # Prepare feature set based on weather availability
        if weather_data:
            features = features[['interaction_distance', 'event_type', 'user_city',
                               'event_city', 'age', 'weather_condition', 'temperature']]
            model = self.weather_model
        else:
            features = features[['interaction_distance', 'event_type', 
                               'user_city', 'event_city', 'age']]
            model = self.no_weather_model
        
        # Create CatBoost pool
        pool = Pool(features, cat_features=features.select_dtypes("object").columns.tolist())
        return model.predict_proba(pool)[:, 1]

    def evaluate(self, y_true, y_score, k_values=[5, 10, 20]):
        metrics = {
            "AUC": roc_auc_score(y_true, y_score),
            "MAP": average_precision_score(y_true, y_score)
        }
        
        sorted_idx = np.argsort(-y_score)
        for k in k_values:
            top_k = y_true[sorted_idx][:k]
            metrics[f"Precision@{k}"] = top_k.mean()
            metrics[f"Recall@{k}"] = top_k.sum() / max(y_true.sum(), 1)
            metrics[f"NDCG@{k}"] = ndcg_score([y_true], [y_score], k=k)
        
        return metrics




In [84]:
# Initialize recommendation system with fixed model loading
rs = RecommendationSystem()

# Get sample user and event data from Hopsworks
user_id = "CD639A"  # Example user from interactions
users_df = rs.users_fv.get_feature_vector({"user_id": user_id}, return_type="pandas")
events_df = rs.events_fv.get_feature_vectors()

# Generate candidates using Two-Tower model
candidates_df = rs.generate_candidates(
    query_model=rs.query_model,
    candidate_model=rs.candidate_model,
    users_df=users_df,
    events_df=events_df,
    k=100
)

# Get weather data (example)
weather_data = {
    'temperature': 18.5,
    'weather_condition': 'sunny'
}

# Rank candidates with weather context
weather_scores = rs.rank_candidates(candidates_df, weather_data=weather_data)

# Rank candidates without weather context
no_weather_scores = rs.rank_candidates(candidates_df)

# Combine results
final_recommendations = candidates_df.copy()
final_recommendations['weather_score'] = weather_scores
final_recommendations['no_weather_score'] = no_weather_scores

# Get ground truth from recent interactions
y_true = rs.interactions_fv.select(["user_id", "event_id", "label"])\
                          .filter(f"user_id = '{user_id}'")\
                          .read()["label"].values

# Calculate metrics
metrics = rs.evaluate(y_true, weather_scores)
print(f"Weather Model Metrics:\n{metrics}")

metrics = rs.evaluate(y_true, no_weather_scores)
print(f"\nNo-Weather Model Metrics:\n{metrics}")

# Display top 5 recommendations
print("\nTop Weather-Based Recommendations:")
print(final_recommendations.sort_values('weather_score', ascending=False).head(5))

print("\nTop Non-Weather Recommendations:")
print(final_recommendations.sort_values('no_weather_score', ascending=False).head(5))


2025-04-20 16:53:43,301 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-20 16:53:43,319 INFO: Initializing external client
2025-04-20 16:53:43,321 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-20 16:53:43,965 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-20 16:53:44,769 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788
2025-04-20 16:53:59,541 WARNING: VersionWarning: No training dataset version was provided to initialise serving. Defaulting to version 1.

2025-04-20 16:54:01,575 WARNING: VersionWarning: No training dataset version was provided to initialise serving. Defaulting to version 1.



FeatureStoreException: The required argument `entries` is missing. If the feature view includes only on-demand features, entries may be left empty or set to None.